In [1]:
#import libraries
import pandas as pd
import os
import zipfile
import shutil

#import torch libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# clone the repo
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
# extract the content
extract_dir = "/content/Train"
os.makedirs(extract_dir, exist_ok=True)

#open and extract the zip file
for i in range(1,3):
  with zipfile.ZipFile(f"/content/Qualcomm-DL-Hackathon/train/images part-{i}.zip","r") as zip_ref:
    zip_ref.extractall(extract_dir)

In [4]:
train_data_csv = pd.read_csv("/content/Qualcomm-DL-Hackathon/train/train.csv")
test = pd.read_csv("/content/Qualcomm-DL-Hackathon/test.csv")

In [5]:
# extract images and split them into train and test

destination_folder = "/content/Train/train_images"

if not os.path.exists(destination_folder):
  os.makedirs(destination_folder)

for i in range(1,3):
  source_folder = f"/content/Train/images part-{i}"
  for root, dirs, files in os.walk(source_folder):
    for file in files:
      src_file = os.path.join(root, file)
      shutil.copy2(src_file, destination_folder)



In [6]:
#splitting datasets into train and test
test_images = list(test["image_names"])
source_folder = f"/content/Train/train_images"
destination_folder = "/content/Train/final_test_images"

if not os.path.exists(destination_folder):
  os.makedirs(destination_folder)
count = 0
for root, dirs, files in os.walk(source_folder):
  for file in files:
    # if count < 1:
    #   print(file, type(file), test_images[0],type(test_images[0]))
    count += 1
    if file in test_images:
      # print("file")
      src_file = os.path.join(root, file)
      shutil.move(src_file, destination_folder)
print(count)

2352


In [7]:
# initialize the res dict to construct final csv at the end
count = 0
res = {"image_names":[], "emergency_or_not":[]}
for root, dirs, files in os.walk(destination_folder):
  for file in files:
    # if count < 1:
    #   print(file, type(file), test_images[0],type(test_images[0]))
    count += 1
    res["image_names"].append(file)
print(count)

706


In [8]:
# Creating a class to handle image to label mapping with(for train) and without labels(for test)
from torch.utils.data import Dataset
from PIL import Image

class CustomImageDatasetWithLabels(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.labels_df.iloc[idx, 0])
        image = Image.open(img_name)
        label = int(self.labels_df.iloc[idx, 1])  # Convert label to int
        if self.transform:
            image = self.transform(image)
        return image, label

class CustomImageDatasetWithoutLabels(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_names = os.listdir(image_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_names[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
# Definie the train and test transforms

train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(20),  # Randomly rotate images by up to 20 degrees
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomVerticalFlip(),  # Randomly flip images vertically
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change brightness, contrast, saturation, and hue
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Randomly translate images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
])


In [10]:

# Paths to your CSV files and image folders
train_csv = '/content/Qualcomm-DL-Hackathon/train/train.csv'
train_image_folder = '/content/Train/train_images'

# Initialize your datasets
train_dataset = CustomImageDatasetWithLabels(csv_file=train_csv, image_folder=train_image_folder, transform=train_transforms)



In [11]:
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_data, test_data = random_split(train_dataset, [train_size, test_size])

In [80]:
# Creating dataloaders

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [98]:
# Initialize the model
from torchvision.models import resnet18
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,2)

model = model.to(device)
# # Set requires_grad=True for all parameters
# for param in model.parameters():
#     param.requires_grad = True

# # Continue as usual
# num_ftrs = model.fc.in_features
# model.fc = nn.Sequential(
#     nn.Dropout(0.5),
#     nn.Linear(num_ftrs, 2)
# )

# model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [82]:
# from torchvision.models import resnet18

# model = resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Sequential(
#     nn.Dropout(0.5),
#     nn.Linear(num_ftrs, 2)
# )

# model = model.to(device)


In [102]:
# Define the optimizer and scehduler
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [116]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    # if (epoch+1) % 10 == 0:
    #   predict_op()
    #   create_csv(f"resnet50_epoch_{epoch+1}")
    scheduler.step()

Epoch 1/5, Loss: 0.0156, Accuracy: 99.47%
Epoch 2/5, Loss: 0.0100, Accuracy: 99.92%
Epoch 3/5, Loss: 0.0154, Accuracy: 99.70%
Epoch 4/5, Loss: 0.0133, Accuracy: 99.85%
Epoch 5/5, Loss: 0.0160, Accuracy: 99.70%


In [78]:
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_data , test_data = random_split(train_dataset, [train_size, test_size])
test_loader_temp = DataLoader(test_data, batch_size=32, shuffle=False)

In [117]:
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 96.97%


In [105]:
# Initialize your test dataset
test_dataset = CustomImageDatasetWithoutLabels(image_folder='/content/Train/final_test_images', transform=test_transforms)
test_loader_temp = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [118]:
res["emergency_or_not"].clear()
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs in test_loader_temp:
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        print([t.item() for t in list(predicted)])
        res["emergency_or_not"].extend([t.item() for t in list(predicted)])

[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0]
[1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1]
[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]
[0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 1, 1, 1, 0, 0, 1,

In [119]:
print(len(res["emergency_or_not"]), len(res["image_names"]))
results = {}
for i in range(len(res["image_names"])):
  results[res["image_names"][i]] = res["emergency_or_not"][i]

706 706


In [121]:
temp = []
import pandas as pd
data = pd.read_csv("/content/Qualcomm-DL-Hackathon/test.csv")
for i in list(data["image_names"]):
  temp.append(results[i])
data["emergency_or_not"] = temp
data.to_csv("submission_e_35.csv", index=False)

In [91]:
results = {}
def predict_op():
  global res
  global results
  res["emergency_or_not"].clear()
  model.eval()
  test_correct = 0
  test_total = 0
  with torch.no_grad():
      for inputs in test_loader_temp:
          inputs = inputs.to(device)

          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          # print([t.item() for t in list(predicted)])
          res["emergency_or_not"].extend([t.item() for t in list(predicted)])
  results = {}
  for i in range(len(res["image_names"])):
    results[res["image_names"][i]] = res["emergency_or_not"][i]


In [73]:
def create_csv(name):
  temp = []
  import pandas as pd
  data = pd.read_csv("/content/Qualcomm-DL-Hackathon/test.csv")
  for i in list(data["image_names"]):
    temp.append(results[i])
  data["emergency_or_not"] = temp
  data.to_csv(f"{name}.csv", index=False)